# Run analysis on multiple files

In [7]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import os
import pyfilaments.analysisutils as analysis
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing

In [15]:
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/analysis_test'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FinerResolution_BendingStiffness'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-06-23'
data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-06-24'
# data_folder = '/home/deepak/Dropbox/LacryModeling/ModellingResults/StochasticActivity/NormalDistribution'


print(os.listdir(data_folder))

# Find all simulation data files and create a list
files_list = []
 # Walk through the folders and identify the simulation data files
for dirs, subdirs, files in os.walk(data_folder, topdown=False):

    root, subFolderName = os.path.split(dirs)

    for fileNames in files:
        if(fileNames.endswith('hdf5') and fileNames[0] != '.'):
            files_list.append(os.path.join(dirs,fileNames))
#         if(fileNames.endswith('hdf5') and fileNames[0] != '.' and fileNames == 'SimResults_00.hdf5'):
#             files_list.append(os.path.join(dirs,fileNames))

print('Simulation files: ', files_list)

def run_filament_analysis(file):
    print('Analyzing file ...')
    print(file)

    filament = analysis.analysisTools(file = file)


    root_folder, file_name = os.path.split(file)

    # Create a data-packet to save for further analysis
    # Collect simulation metadata
    metadata_path = os.path.join(root_folder, 'metadata.csv')
    assert(os.path.exists(metadata_path))
    df = pd.read_csv(metadata_path)
    
    periodic_flag = None 
    min_period = None
    
    filament.filament_tip_coverage(save = True)
    # Check if simulation was completed or terminated before completion
    if(int(filament.Time[-1]) == int(df[' activity time scale']*500)):
        simulation_completed = True
        # Classify the dynamics
        periodic_flag, min_period = filament.classify_filament_dynamics()
    else:
        simulation_completed = False

    df['simulation completed'] = simulation_completed
    df['periodic dynamics'] = periodic_flag
    df['period'] = min_period
    df['max unique locations'] = filament.derived_data['unique position count'][-1]
    
    save_file = file_name[:-5] + '_analysis.csv'
    save_folder = os.path.join(root_folder, 'Analysis')

    if(not os.path.exists(save_folder)):
        os.makedirs(save_folder)
    df.to_csv(os.path.join(save_folder, save_file))




['SimResults_Np_32_Shape_line_kappa_hat_6_k_24.89_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_25.0_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_24.44_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_24.78_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_24.33_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_24.56_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_24.22_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_24.11_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_24.67_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_24.0_b0_2.1_F_0_S_0_D_1.5_acti

In [16]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

results = Parallel(n_jobs=num_cores,  verbose=10)(delayed(run_filament_analysis)(file) for file in tqdm(files_list))


100%|██████████| 30/30 [00:08<00:00,  3.65it/s]
[Parallel(n_jobs=12)]: Done  11 out of  30 | elapsed:   21.2s remaining:   36.6s
[Parallel(n_jobs=12)]: Done  15 out of  30 | elapsed:   23.0s remaining:   23.0s
[Parallel(n_jobs=12)]: Done  19 out of  30 | elapsed:   24.4s remaining:   14.1s
[Parallel(n_jobs=12)]: Done  23 out of  30 | elapsed:   28.4s remaining:    8.6s
[Parallel(n_jobs=12)]: Done  27 out of  30 | elapsed:   33.2s remaining:    3.7s
[Parallel(n_jobs=12)]: Done  30 out of  30 | elapsed:   34.7s finished
